In [6]:
import csv
import tensorflow as tf
import numpy as np
import os
import pathlib
import shutil
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

FAST_RUN = False

epochs=1000
batch_size = 128
target_size = (180, 180)
threshold = 0.50
input_path = "./kaggle/input/plant-pathology-2021-fgvc8/"
checkpoint_filepath = './tmp/checkpoint'
model_filepath = './model'
class_names = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
num_classes = len(class_names)

def labels_to_str(vector):
    result = []
    for idx, v in enumerate(vector):
        if v == 1:
            result.append(class_names[idx])
    return " ".join(result)


print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
model = tf.keras.models.load_model(model_filepath)

In [8]:
test_df = pd.read_csv(input_path + 'sample_submission.csv')
test_path = input_path + 'test_images'

image_test_datagen = ImageDataGenerator(
    rescale=1./255,
)

test_generator = image_test_datagen.flow_from_dataframe(
    test_df,
    directory = test_path,
    x_col = "image",
    y_col = "labels",
    target_size = target_size,
    color_mode="rgb",
    batch_size=1,
    shuffle=False,
    seed=123,
    subset=None
)

Found 3 validated image filenames belonging to 1 classes.


In [9]:
predicts = model.predict(test_generator)
verdict = (predicts>threshold)
answer = []
for i in range(verdict.shape[0]):
    answer.append(labels_to_str(verdict[i]))


test_df['labels'] = np.array(answer)
print(test_df)
test_df.to_csv('submission.csv', index=False)

3/3 [==============================] - 5s 30ms/step
                  image labels
0  85f8cb619c66b863.jpg   rust
1  ad8770db05586b59.jpg       
2  c7b03e718489f3ca.jpg       
